%pip install kaggle

import kaggle
path = kaggle.dataset_download("waleedumer/egyptian-hieroglyphics-datasets")

print(path)

In [5]:

%pip install torchvision

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 13.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from torch import nn

# Import torchvision 
import torchvision
from torchvision import datasets as ds
from torchvision.transforms import ToTensor as toTvTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt
import numpy

In [7]:
train_data = ds.FashionMNIST(
    root = "../dat/",
    train = True, # get training data
    download = True, # download data if it doesn't exist on disk
    transform = toTvTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

test_data = ds.FashionMNIST(
    root = "../dat/",
    train=False, # get test data
    download=True,
    transform=toTvTensor()
)

100%|██████████| 26421880/26421880 [00:04<00:00, 5631241.80it/s]


Extracting ../dat/FashionMNIST\raw\train-images-idx3-ubyte.gz to ../dat/FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 984327.10it/s]


Extracting ../dat/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../dat/FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5417082.28it/s]


Extracting ../dat/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../dat/FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 5130025.42it/s]

Extracting ../dat/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../dat/FashionMNIST\raw



In [9]:
DEVICE = "cpu"
classes = ds.FashionMNIST.classes

In [17]:
class ConvNet(nn.Module):  
    def __init__(self, in_channels: int, hidden_layer_nodes: int, out_channels: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=hidden_layer_nodes,
                      kernel_size=3, # how big is the square that's going over the image?
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_layer_nodes,
                      out_channels=hidden_layer_nodes,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_layer_nodes, hidden_layer_nodes, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_layer_nodes, hidden_layer_nodes, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.out = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from?
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_layer_nodes*7*7,
                      out_features=out_channels)
        )

    def forward(self, x: torch.Tensor):
        return self.out(self.block_2(self.block_1(x)))

In [18]:
def train(model: torch.nn.Module,
            data_loader: torch.utils.data.DataLoader,
            loss_fn: torch.nn.Module,
            optimizer: torch.optim.Optimizer,
            accuracy_fn,
            device: torch.device = "cuda") -> float:
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")
    return train_acc

def test(data_loader: torch.utils.data.DataLoader, 
         model: torch.nn.Module,
         loss_fn: torch.nn.Module,
         accuracy_fn,
        device: torch.device = 'cuda') -> float:
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
        return test_acc 

In [19]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
# Setup the batch size hyperparameter

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(
    train_data, # dataset to turn into iterable
    batch_size=BATCH_SIZE, # how many samples per batch? 
    shuffle=True # shuffle data every epoch?
)

test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False # don't necessarily have to shuffle the testing data
)

In [20]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [25]:
net = ConvNet(
    1,
    10,
    len(classes)
)
net.to(DEVICE)

ConvNet(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [29]:
EPOCHS = 3
list_train_acc = []
list_test_acc = []

In [30]:
import tqdm


for i in tqdm.tqdm(range(EPOCHS)):
   print(f"epoch: {i+1}\n--------------------")
   train_acc = train(
       net,
       train_dataloader,
       nn.CrossEntropyLoss(),
       torch.optim.SGD(net.parameters(), lr = 0.1),
       accuracy_fn,
       DEVICE
    )
   
   if (i + 1)%1 == 0: 
      test_acc = test(
         test_dataloader,
         net,
         nn.CrossEntropyLoss(),
         accuracy_fn,
         DEVICE
      )
      print("test data logged")
   
   list_train_acc.append(train_acc)
   list_test_acc.append(test_acc)

  0%|          | 0/3 [00:00<?, ?it/s]

epoch: 1
--------------------
Train loss: 0.25312 | Train accuracy: 90.79%


 33%|███▎      | 1/3 [00:21<00:42, 21.26s/it]

Test loss: 0.31025 | Test accuracy: 88.45%

test data logged
epoch: 2
--------------------
Train loss: 0.24573 | Train accuracy: 91.06%


 67%|██████▋   | 2/3 [00:41<00:20, 20.75s/it]

Test loss: 0.29198 | Test accuracy: 89.64%

test data logged
epoch: 3
--------------------
Train loss: 0.24283 | Train accuracy: 91.18%


100%|██████████| 3/3 [01:00<00:00, 20.18s/it]

Test loss: 0.28971 | Test accuracy: 89.60%

test data logged
